# Week 9 - serverless

In [1]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.13.0


In [2]:
model = keras.models.load_model('bees-wasps.h5')

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

* 21 Mb
* 43 Mb
* 80 Mb
* 164 Mb

In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\claim\AppData\Local\Temp\tmpcpb_lqt8\assets


INFO:tensorflow:Assets written to: C:\Users\claim\AppData\Local\Temp\tmpcpb_lqt8\assets


In [9]:
!ls -hl

total 129M
-rw-r--r-- 1 claim 197609  86M Dec  3 22:05 bees-wasps.h5
-rw-r--r-- 1 claim 197609 2.4K Dec  4 00:24 homework.ipynb
-rw-r--r-- 1 claim 197609  43M Dec  4 00:24 model.tflite


We observe that the converted model is 43M

In [10]:
import tensorflow.lite  as tflite

In [11]:
interpreter = tflite.Interpreter(model_path='./model.tflite')
interpreter.allocate_tensors()

## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

* 3
* 7
* 13
* 24

In [12]:
interpreter.get_input_details()

[{'name': 'serving_default_conv2d_input:0',
  'index': 0,
  'shape': array([  1, 150, 150,   3]),
  'shape_signature': array([ -1, 150, 150,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [13]:
input_index, output_index = interpreter.get_input_details()[0]['index'], interpreter.get_output_details()[0]['index']
input_index, output_index

(0, 13)

Output index is 13

In [14]:
!pip install pillow

In [16]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

url = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'
img = download_image(url)
img_resized = prepare_image(img, (150, 150))

C:\Users\claim\AppData\Local\Temp\ipykernel_19328\4105156323.py:17: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


## Question 3

Now we need to turn the image into numpy array and pre-process it. 

After the pre-processing, what's the value in the first pixel, the R channel?

* 0.3450980
* 0.5450980
* 0.7450980
* 0.9450980


In [17]:
import numpy as np

In [35]:
def prepare_input(img):
    return img * (1. / 255)

img_array = np.float32([np.array(img_resized)])
img_preprocessed = prepare_input(img_array)
img_preprocessed[0][0][0][0]

0.9450981

The value of first pixel in R channel is `0.9451`.

## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.258
* 0.458
* 0.658
* 0.858


In [36]:
interpreter.set_tensor(input_index, img_preprocessed)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds

array([[0.65898407]], dtype=float32)

Output is `0.658`.

## Question 5

Download the base image `agrigorev/zoomcamp-bees-wasps:v2`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 162 Mb
* 362 Mb
* 662 Mb
* 962 Mb

## Answer  

We run the docker command 
```shell
docker image pull agrigorev/zoomcamp-bees-wasps:v2
docker images
```
We see that the size is 662Mb.

## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `bees-wasps-v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference). 
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg

What's the output from the model?

* 0.2453
* 0.4453
* 0.6453
* 0.8453

Using the Dockerfile, we will build a container image that will run the Python app `test.py`. 
```docker
docker build -t zoomcamp-bees-wasps .
docker run -it --rm -p 8080:8080 zoomcamp-bees-wasps
```
After running the image, we can use the app to get predictions for an image. 
Open a new window and wun `python test.py`. 

For the given image, it's `0.4453350603580475`. 
